In [1]:
%pip install --upgrade google-cloud-firestore
%pip install pandas
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from google.cloud import firestore
db = firestore.Client(project='de1000-dev-mwc-ran-agent', database="ran-guardian-event-scout")

In [27]:
import pandas as pd
df = pd.read_csv("germany_locations.csv")
df.head()

,priority,collection_id
0,2,Aach Baden-Württemberg
1,2,Aach Rheinland-Pfalz
2,1,Aachen Nordrhein-Westfalen
3,2,Aalen Baden-Württemberg
4,2,Aarbergen Hessen


In [28]:


# Function to map priority values to string labels
def map_priority(priority_value):
  """Maps numerical priority to string representation.

  Args:
      priority_value: The numerical priority value (e.g., 1.0, 2.0).

  Returns:
      A string representing the priority level ('High', 'Medium', or 'Low').
  """
  if priority_value == 1.0:
    return 'high'
  elif priority_value == 2.0:
    return 'medium'
  else:
      return 'low'

# Apply the mapping function to the 'priority' column
df['priority'] = df['priority'].apply(map_priority)

In [29]:
samples = df[0:2]
samples

,priority,collection_id
0,medium,Aach Baden-Württemberg
1,medium,Aach Rheinland-Pfalz


In [30]:
import datetime
for index, location in df.iterrows():
    
    coll_id = location["collection_id"]
    coll_id = coll_id.replace("/", " ")
    print(index, coll_id)
    doc_ref = db.collection("locations").document(coll_id)
    doc_ref.set({"num_events": 0, "priority": location["priority"], u"last_scanned": datetime.datetime.fromtimestamp(0, tz=datetime.timezone.utc)})

    doc_ref_stats = db.collection("locations").document("0_stats")
    doc_ref_stats.update({
        "num_locations": firestore.Increment(1)
    })

0 Aach Baden-Württemberg


NotFound: 404 No document to update: projects/de1000-dev-mwc-ran-agent/databases/ran-guardian-event-scout/documents/locations/stats